In [69]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from IPython.display import display, HTML
import numpy as np
import json
from scipy import stats
import math
import plotly.graph_objects as go

from datetime import datetime
#plt.rcParams["figure.figsize"] = (17, 5) # (w, h)

In [70]:
root = '../data_storage/register'
file_names = []
dates = []

for subdir, dirs, files in os.walk(root):
    for file in files:
        file_names.append(os.path.join(subdir, file).replace('\\', '/'))
        
        datetime_object = datetime.strptime(file[5:-9], '%d %b %Y %H:%M:%S')
        dates.append(datetime_object)
        
youngest = max(dates)

for file in files:
    datetime_object = datetime.strptime(file[5:-9], '%d %b %Y %H:%M:%S')
    if(datetime_object == youngest):
        youngest = '../data_storage/' + file

In [72]:
main_frame = pd.DataFrame()

for f in file_names:
    df = pd.read_json(f)
    df['created_at'] = pd.to_datetime(f[30:-9])
    main_frame = main_frame.append(df)

main_frame['updated'] = pd.to_datetime(main_frame['updated'])
main_frame = main_frame[main_frame['updated'].dt.year>2019]


#Get dummy Variables
dfDummiesHigh = pd.get_dummies(main_frame['icu_high_care'], prefix = 'high_care')
dfDummiesLow = pd.get_dummies(main_frame['icu_low_care'], prefix = 'low_care')
dfDummiesEcmo = pd.get_dummies(main_frame['ecmo'], prefix = 'ecmo')
main_frame = pd.concat([main_frame, dfDummiesHigh, dfDummiesLow, dfDummiesEcmo], axis=1)

# Get youngest frame
youngest_frame = main_frame[main_frame['created_at']==main_frame['created_at'].max()]

In [73]:
main_frame.head(3)

,hospital,contact,fed,icu_low_care,icu_high_care,updated,ecmo,created_at,high_care_green,high_care_red,high_care_yellow,low_care_green,low_care_red,low_care_yellow,ecmo_green,ecmo_red,ecmo_yellow
0,"Kreisklinik Wolfratshausen, Moosbauerweg 5, 82...",Website,BY,green,green,2020-03-28 08:48:00,NaN,2020-03-28 09:13:28,1,0,0,1,0,0,0,0,0
1,"Klinikum-Bremen-Nord, Klinik für Anästhesiolog...",Intensivstation Website,HB,green,green,2020-03-27 18:20:00,NaN,2020-03-28 09:13:28,1,0,0,1,0,0,0,0,0
2,"Helios Frankenwaldklinik Kronach, Anästhesie, ...",4a Website,BY,yellow,yellow,2020-03-27 17:00:00,NaN,2020-03-28 09:13:28,0,0,1,0,0,1,0,0,0


In [74]:
#main_frame.describe()
#main_frame['fed'].value_counts()
main_frame[main_frame['fed']=='0']

,hospital,contact,fed,icu_low_care,icu_high_care,updated,ecmo,created_at,high_care_green,high_care_red,high_care_yellow,low_care_green,low_care_red,low_care_yellow,ecmo_green,ecmo_red,ecmo_yellow
494,"Klinikum Lüdenscheid, Paulmannshöher Straße 14...",Website,0,green,green,2020-03-25 14:02:00,NaN,2020-03-25 19:13:31,1,0,0,1,0,0,0,0,0
483,"Klinikum Lüdenscheid, Paulmannshöher Straße 14...",Website,0,green,green,2020-03-25 14:02:00,NaN,2020-03-25 13:57:08,1,0,0,1,0,0,0,0,0
483,"Klinikum Lüdenscheid, Paulmannshöher Straße 14...",Website,0,green,green,2020-03-25 14:02:00,NaN,2020-03-25 14:13:29,1,0,0,1,0,0,0,0,0
494,"Klinikum Lüdenscheid, Paulmannshöher Straße 14...",Website,0,green,green,2020-03-25 14:02:00,NaN,2020-03-25 19:18:20,1,0,0,1,0,0,0,0,0
483,"Klinikum Lüdenscheid, Paulmannshöher Straße 14...",Website,0,green,green,2020-03-25 14:02:00,NaN,2020-03-25 14:02:46,1,0,0,1,0,0,0,0,0


In [75]:
youngest_frame.head()

,hospital,contact,fed,icu_low_care,icu_high_care,updated,ecmo,created_at,high_care_green,high_care_red,high_care_yellow,low_care_green,low_care_red,low_care_yellow,ecmo_green,ecmo_red,ecmo_yellow
0,"Sana Dreifaltigkeits-Krankenhaus Köln, Anästhe...",Intensivstation Website,NRW,red,red,2020-03-29 11:39:00,NaN,2020-03-29 17:07:03,0,1,0,0,1,0,0,0,0
1,"Elbe-Elster-Klinikum, Standort Finsterwalde, A...",Intensivtation Standort Finsterwalde Website,BB,green,green,2020-03-28 15:24:00,NaN,2020-03-29 17:07:03,1,0,0,1,0,0,0,0,0
2,Medizin Campus Bodensee - Klinikum Friedrichsh...,Website,BW,green,green,2020-03-29 13:54:00,NaN,2020-03-29 17:07:03,1,0,0,1,0,0,0,0,0
3,"Krankenhaus Bethel Berlin, Anästhesie und Inte...",ITS Website,BE,green,green,2020-03-29 14:29:00,NaN,2020-03-29 17:07:03,1,0,0,1,0,0,0,0,0
4,"Christophoruskliniken Coesfeld, Südring 41, 48...",cD1 Interdisziplinäre Intensivstation Website,NRW,green,green,2020-03-27 16:42:00,NaN,2020-03-29 17:07:03,1,0,0,1,0,0,0,0,0


In [76]:
df = main_frame.groupby(['created_at']).sum()
df = df.sort_values('created_at')
df.head(5)

,high_care_green,high_care_red,high_care_yellow,low_care_green,low_care_red,low_care_yellow,ecmo_green,ecmo_red,ecmo_yellow
created_at,,,,,,,,,
2020-03-25 12:13:28,502.0,38.0,102.0,474.0,67.0,101.0,119.0,47.0,24.0
2020-03-25 13:57:08,505.0,38.0,108.0,475.0,69.0,107.0,120.0,47.0,25.0
2020-03-25 14:02:46,504.0,37.0,110.0,475.0,69.0,107.0,121.0,45.0,25.0
2020-03-25 14:13:29,504.0,37.0,110.0,476.0,69.0,106.0,121.0,45.0,25.0
2020-03-25 19:13:31,513.0,37.0,109.0,482.0,69.0,108.0,124.0,44.0,25.0


In [77]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.high_care_green, 
                         name="Status: Green", mode = 'lines+markers',
                         line_color='SeaGreen',connectgaps=False))
fig.add_trace(go.Scatter(x=df.index, y=df.high_care_yellow, 
                         name="Status: Yellow", mode = 'lines+markers',
                         line_color='DarkOrange',connectgaps=False))
fig.add_trace(go.Scatter(x=df.index, y=df.high_care_red, 
                         name="Status: Red",mode = 'lines+markers',
                         line_color='Crimson',connectgaps=False))

fig.update_layout(title_text='Status of High care', xaxis_rangeslider_visible=True, template='simple_white')
fig.show()
fig.write_html("high_care.html")

In [78]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.low_care_green, 
                         name="Status: Green", mode = 'lines+markers',
                         line_color='SeaGreen',connectgaps=False))
fig.add_trace(go.Scatter(x=df.index, y=df.low_care_yellow, 
                         name="Status: Yellow", mode = 'lines+markers',
                         line_color='DarkOrange',connectgaps=False))
fig.add_trace(go.Scatter(x=df.index, y=df.low_care_red, 
                         name="Status: Red",mode = 'lines+markers',
                         line_color='Crimson',connectgaps=False))

fig.update_layout(title_text='Status of Low care', xaxis_rangeslider_visible=True, template='simple_white')
fig.show()
fig.write_html("low_care.html")

In [79]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.ecmo_green, 
                         name="Status: Green", mode = 'lines+markers',
                         line_color='SeaGreen',connectgaps=False))
fig.add_trace(go.Scatter(x=df.index, y=df.ecmo_yellow, 
                         name="Status: Yellow", mode = 'lines+markers',
                         line_color='DarkOrange',connectgaps=False))
fig.add_trace(go.Scatter(x=df.index, y=df.ecmo_red, 
                         name="Status: Red",mode = 'lines+markers',
                         line_color='Crimson',connectgaps=False))

fig.update_layout(title_text='Status of ECMO', xaxis_rangeslider_visible=True, template='simple_white')
fig.show()
fig.write_html("low_care.html")

In [80]:
from tqdm import tqdm
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import ArcGIS
tqdm.pandas()

locator = ArcGIS(token_lifetime=20, timeout=5)
#location = locator.geocode('Diakonissenkrankenhaus Flensburg, Anästhesie und Innere Medizin, Knuthstr 1, 24939 Flensburg')
#print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

#1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1,max_retries=5)
#2- - create location column
youngest_frame['location'] = youngest_frame['hospital'].progress_apply(geocode)
# 3 - create longitude, latitude and altitude from location column (returns tuple)
youngest_frame['point'] = youngest_frame['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
youngest_frame[['latitude', 'longitude', 'altitude']] = pd.DataFrame(youngest_frame['point'].tolist(), 
                                                                     index=youngest_frame.index)

/Users/andreasfussstetter/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

100%|██████████| 729/729 [19:43<00:00,  1.62s/it]
/Users/andreasfussstetter/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/andreasfussstetter/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [81]:
youngest_frame.describe()

,high_care_green,high_care_red,high_care_yellow,low_care_green,low_care_red,low_care_yellow,ecmo_green,ecmo_red,ecmo_yellow,latitude,longitude,altitude
count,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.0
mean,0.784636,0.061728,0.153635,0.748971,0.101509,0.149520,0.168724,0.048011,0.041152,51.058341,9.691115,0.0
std,0.411357,0.240827,0.360846,0.433903,0.302209,0.356845,0.374765,0.213936,0.198779,1.738263,4.513479,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.766600,-94.856070,0.0
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.716880,7.831640,0.0
50%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.235500,9.502750,0.0
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.358657,11.727820,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,54.925610,14.971013,0.0


In [82]:
fig = go.Figure(data=go.Scattergeo(
        lon = youngest_frame['longitude'],
        lat = youngest_frame['latitude'],
        text = youngest_frame['hospital'],
        mode = 'markers',
        marker_color = youngest_frame['icu_high_care'],
        ))

fig.update_layout(
        title = 'Hospitals',
        geo_scope='world',
    )
fig.show()

In [85]:
mapbox_access_token = 'pk.eyJ1IjoibWFwc3RldHRlciIsImEiOiJjazhkY2NvbnEwdW54M2dwZWdqdnQyaTExIn0.zE90Zn874T8f2jGAGMubvQ'

fig = go.Figure(go.Scattermapbox(
        lon = youngest_frame['longitude'],
        lat = youngest_frame['latitude'],
        text = youngest_frame['hospital'],
        mode = 'markers',
        marker_color = youngest_frame['icu_high_care']))

fig.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=52,
            lon=9
        ),
        zoom= 4
    ))


fig.show()
fig.write_html("map.html")